# Hotel revenue management

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
hotel = pd.read_csv('hotel-probability.csv')

In [4]:
hotel['forecast'] = hotel['probability'] * 540

In [5]:
hotel

,r,din,dout,probability,forecast
0,Q,1,2,0.018519,10.0
1,Q,1,3,0.011111,6.0
2,Q,1,4,0.011111,6.0
3,Q,1,5,0.007407,4.0
4,Q,1,6,0.007407,4.0
...,...,...,...,...,...
79,C,5,7,0.003704,2.0
80,C,5,8,0.003704,2.0
81,C,6,7,0.003704,2.0
82,C,6,8,0.003704,2.0


# Part 1: Understanding the data
### a)

In [6]:
hotel[hotel['r'] == 'Q'].sum()['probability']

0.3222222239999999

### b)

In [7]:
hotel[(hotel['din'] == 4) & (hotel['dout'] == 7)].sum()['probability']

0.014814815

### c)

In [8]:
1 - hotel.sum()['probability']

0.259259247

### d)

In [32]:
hotel[(hotel['r'] == 'C') & (hotel['din'] == 5) & (hotel['dout'] == 6)]['probability'] * 540

78    6.0
Name: probability, dtype: float64

# Part 2: Determining an optimal static allocation
### b)

In [10]:
for i in range(1,9):
    hotel[str(i)] = 0

In [11]:
for i in range(hotel.shape[0]):
    colin = hotel.iloc[i]['din']
    colout = hotel.iloc[i]['dout']
    for j in range(colin, colout):
        hotel[str(j)][i] = 1

In [12]:
hotel['rev'] = 0.0
pricepernight = [[200, 200, 200, 230, 230, 230, 200, 200],
                [250, 250, 250, 287.5, 287.5, 287.5, 250, 250],
                [300, 300, 300, 345, 345, 345, 300, 300]]
for i in range(hotel.shape[0]):
    if hotel.iloc[i]['r'] == 'Q':
        hotel['rev'][i] = sum(hotel.iloc[i, 5:13] * pricepernight[0])
    elif hotel.iloc[i]['r'] == 'K':
        hotel['rev'][i] = sum(hotel.iloc[i, 5:13] * pricepernight[1])
    elif hotel.iloc[i]['r'] == 'C':
        hotel['rev'][i] = sum(hotel.iloc[i, 5:13] * pricepernight[2])

In [13]:
hotel

,r,din,dout,probability,forecast,1,2,3,4,5,6,7,8,rev
0,Q,1,2,0.018519,10.0,1,0,0,0,0,0,0,0,200.0
1,Q,1,3,0.011111,6.0,1,1,0,0,0,0,0,0,400.0
2,Q,1,4,0.011111,6.0,1,1,1,0,0,0,0,0,600.0
3,Q,1,5,0.007407,4.0,1,1,1,1,0,0,0,0,830.0
4,Q,1,6,0.007407,4.0,1,1,1,1,1,0,0,0,1060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,C,5,7,0.003704,2.0,0,0,0,0,1,1,0,0,690.0
80,C,5,8,0.003704,2.0,0,0,0,0,1,1,1,0,990.0
81,C,6,7,0.003704,2.0,0,0,0,0,0,1,0,0,345.0
82,C,6,8,0.003704,2.0,0,0,0,0,0,1,1,0,645.0


In [53]:
from gurobipy import * 

n = hotel.shape[0]
m = Model()
x = m.addVars(n, lb = 0, ub = hotel['forecast'])

# Add Room Capacity Constraints
room_capacity_constrs_Q = {}
room_capacity_constrs_K = {}
room_capacity_constrs_C = {}
for i in range(1,9):
    room_capacity_constrs_Q[i-1] = m.addConstr(sum(x[j] * hotel[str(i)][j] for j in range(0, 28)) <= 50)
    room_capacity_constrs_K[i-1] = m.addConstr(sum(x[j] * hotel[str(i)][j] for j in range(28, 56)) <= 50)
    room_capacity_constrs_C[i-1] = m.addConstr(sum(x[j] * hotel[str(i)][j] for j in range(56, 84)) <= 20)

# Objective:
print("Creating objective:")
m.setObjective(sum(hotel['rev'][i] * x[i] for i in range(n)), GRB.MAXIMIZE)

# Update and solve
m.update()

m.optimize()

print('optimal revenue is:', m.objval)

Creating objective:
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 24 rows, 84 columns and 252 nonzeros
Model fingerprint: 0x73149771
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 2e+03]
  Bounds range     [2e+00, 4e+01]
  RHS range        [2e+01, 5e+01]
Presolve removed 12 rows and 16 columns
Presolve time: 0.01s
Presolved: 12 rows, 68 columns, 159 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1148000e+05   1.135000e+02   0.000000e+00      0s
Extra simplex iterations after uncrush: 1
      16    1.7214000e+05   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.01 seconds
Optimal objective  1.721400018e+05
optimal revenue is: 172140.0017788


### c)

In [54]:
x_opt = [x[i].x for i in range(n)]
x_opt = pd.DataFrame(x_opt)

In [118]:
index = x_opt.sort_values(by = 0, ascending = False).head().index

In [121]:
hotel.iloc[index, 0:4]

,r,din,dout,probability
35,K,2,3,0.037037
41,K,3,4,0.037037
46,K,4,5,0.029630
0,Q,1,2,0.018519
36,K,2,4,0.018519


### d)

In [57]:
print( [room_capacity_constrs_Q[i].pi for i in range(7)])
print( [room_capacity_constrs_K[i].pi for i in range(7)])
print( [room_capacity_constrs_C[i].pi for i in range(7)])

[0.0, 200.0, 200.0, 230.0, 230.0, 0.0, 0.0]
[0.0, 250.0, 250.0, 287.5, 0.0, 0.0, 0.0]
[0.0, 300.0, 300.0, 345.0, 345.0, 345.0, 0.0]


### e)

In [58]:
10 * (200 + 200 + 230 + 230) - 10 * (250 + 250 + 287.5) 

725.0

# Part 3: Determining an optimal dynamic allocation policy

### a)

In [59]:
# initial values
B = np.array([[50, 50, 50, 50, 50, 50, 50],
            [50, 50, 50, 50, 50, 50, 50],
            [20, 20, 20, 20, 20, 20, 20]])
T = 540

In [60]:
m.Params.outputflag = 0

probability_aug = np.zeros(n+1)
probability_aug[0:n] = hotel['probability'] 
probability_aug[n] = 1 - sum(hotel['probability']) # Last element is one minus the rest.


#simulation
nSimulations = 100
np.random.seed(50)


results_revenue = np.zeros(nSimulations)
results_remaining_room_Q = np.zeros( (nSimulations, 7) )
results_remaining_room_K = np.zeros( (nSimulations, 7) )
results_remaining_room_C = np.zeros( (nSimulations, 7) )


# loop
for s in range(nSimulations):
    total_revenue = 0.0
    b = B.copy() 
    arrival_sequence = np.random.choice(range(n+1), T, p=probability_aug)
    for t in range(T):
        # Stop if all room have been sold:
        if ((b == 0).all()):
            break
        
        i = arrival_sequence[t]
        if (i < n):
            if (i in range(0,28)):
                idx = []
                for idx_X in range(5,12):
                    if hotel.iloc[i,idx_X] != 0:
                        idx.append(idx_X-5)
                if ((b[0]* hotel.iloc[i,5:12])[idx]>0).all():
                    b[0][idx] = b[0][idx]-1
                    total_revenue += hotel['rev'][i]
                    
            elif (i in range(28, 56)):
                idx = []
                for idx_X in range(5,12):
                    if hotel.iloc[i,idx_X] != 0:
                        idx.append(idx_X-5)
                if ((b[1]* hotel.iloc[i,5:12])[idx]>0).all():
                    b[1][idx] = b[1][idx]-1
                    total_revenue += hotel['rev'][i]
                    
            elif (i in range(56, 84)):
                idx = []
                for idx_X in range(5,12):
                    if hotel.iloc[i,idx_X] != 0:
                        idx.append(idx_X-5)
                if ((b[2]* hotel.iloc[i,5:12])[idx]>0).all():
                    b[2][idx] = b[2][idx]-1
                    total_revenue += hotel['rev'][i]       
                    

    results_revenue[s] = total_revenue
mean_LP_revenue = results_revenue.mean()

print("Mean LP policy revenue: ", mean_LP_revenue)

Mean LP policy revenue:  147472.025


### b)

In [61]:
[room_capacity_constrs_K[i].pi for i in range(7)]

[0.0, 250.0, 250.0, 287.5, 0.0, 0.0, 0.0]

In [62]:
250 + 250 + 287.5

787.5

### c)

In [64]:
# setting up
def bpc(b,t):
    for i in range(7):
        room_capacity_constrs_Q[i].rhs = b[0][i]
    
    for i in range(7):
        room_capacity_constrs_K[i].rhs = b[1][i]
    
    for i in range(7):
        room_capacity_constrs_C[i].rhs = b[2][i]
    
    for i in range(n):
        x[i].ub = (T - t)* hotel['probability'][i]
        
    m.update()
    m.optimize()
    
    dual_val = []
    dual_val.append([room_capacity_constrs_Q[i].pi for i in range(7)])
    dual_val.append([room_capacity_constrs_K[i].pi for i in range(7)])
    dual_val.append([room_capacity_constrs_C[i].pi for i in range(7)])
    
    return dual_val

m.Params.outputflag = 0

probability_aug = np.zeros(n+1)
probability_aug[0:n] = hotel['probability'] 
probability_aug[n] = 1 - sum(hotel['probability']) # Last element is one minus the rest.


#simulation
nSimulations = 100
np.random.seed(50)


results_revenue = np.zeros(nSimulations)
results_remaining_room_Q = np.zeros( (nSimulations, 7) )
results_remaining_room_K = np.zeros( (nSimulations, 7) )
results_remaining_room_C = np.zeros( (nSimulations, 7) )


# loop
for s in range(nSimulations):
    total_revenue = 0.0
    b = B.copy() 
    arrival_sequence = np.random.choice(range(n+1), T, p=probability_aug)
    for t in range(T):
        # Stop if all room have been sold:
        if ((b == 0).all()):
            break
        
        i = arrival_sequence[t]
        if (i < n):
            dual_val = bpc(b,t)
            # Compute the total bid price of the request:
            if i in range(0,28):
                total_bid_price = sum(dual_val[0] * hotel.iloc[i,5:12])
            elif i in range(28,56):
                total_bid_price = sum(dual_val[1] * hotel.iloc[i,5:12])
            elif i in range(56,84):
                total_bid_price = sum(dual_val[2] * hotel.iloc[i,5:12])
                
            # If the revenue is at least the total bid price, and there is at least one
            # seat on each leg of this itinerary ...* hotel.iloc[i,5:13]
            if hotel['rev'][i] >= total_bid_price:
                if (i in range(0,28)):
                    idx = []
                    for idx_X in range(5,12):
                        if hotel.iloc[i,idx_X] != 0:
                            idx.append(idx_X-5)
                    if ((b[0]* hotel.iloc[i,5:12])[idx]>0).all():
                        b[0][idx] = b[0][idx]-1
                        total_revenue += hotel['rev'][i]
                    
                elif (i in range(28, 56)):
                    idx = []
                    for idx_X in range(5,12):
                        if hotel.iloc[i,idx_X] != 0:
                            idx.append(idx_X-5)
                    if ((b[1]* hotel.iloc[i,5:12])[idx]>0).all():
                        b[1][idx] = b[1][idx]-1
                        total_revenue += hotel['rev'][i]
                    
                elif (i in range(56, 84)):
                    idx = []
                    for idx_X in range(5,12):
                        if hotel.iloc[i,idx_X] != 0:
                            idx.append(idx_X-5)
                    if ((b[2]* hotel.iloc[i,5:12])[idx]>0).all():
                        b[2][idx] = b[2][idx]-1
                        total_revenue += hotel['rev'][i]       

    results_revenue[s] = total_revenue
mean_LP_revenue = results_revenue.mean()

print("Mean LP policy revenue: ", mean_LP_revenue)

Mean LP policy revenue:  163644.975


### d)

In [113]:
(unique, counts) = np.unique(arrival_sequence, return_counts=True)
frequencies = pd.DataFrame(np.asarray((unique, counts)).T)
index = frequencies.sort_values(by = 1, ascending= False).head(11)[1:11][0]

In [115]:
hotel.iloc[index, 0:4]

,r,din,dout,probability
13,Q,3,4,0.074074
8,Q,2,4,0.037037
41,K,3,4,0.037037
35,K,2,3,0.037037
0,Q,1,2,0.018519
64,C,2,4,0.018519
46,K,4,5,0.029630
36,K,2,4,0.018519
14,Q,3,5,0.014815
1,Q,1,3,0.011111
